# 使用 Selenium 自動化取得 Google Cookies

這個 Notebook 將引導您完成以下步驟：
1.  安裝必要的 Python 套件 (`selenium`)。
2.  啟動一個由程式碼控制的 Chrome 瀏覽器。
3.  在該瀏覽器中手動登入您的 Google 帳號。
4.  程式會自動偵測您登入成功，並將登入後的 Cookies 儲存為一個名為 `google_maps_cookies.json` 的檔案。

**事前準備：**
*   您的電腦需要安裝 Google Chrome 瀏覽器。
*   `selenium` 會嘗試自動下載與您 Chrome 版本對應的 `chromedriver`。如果失敗，您可能需要手動下載並將其路徑加入到系統環境變數中。

In [ ]:
# 執行此儲存格來安裝 selenium 套件
# 如果您是第一次安裝，安裝完成後可能需要重啟 Jupyter 核心 (Kernel)
%pip install selenium


In [1]:
import json
import logging
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# --- 設定 ---
# 設定日誌紀錄器
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
_LOGGER = logging.getLogger(__name__)

# 設定儲存 Cookies 的檔案路徑 (預設儲存在 Notebook 相同目錄下)
COOKIE_PATH = "google_maps_cookies.json"

# 登入操作的超時時間 (秒)
LOGIN_TIMEOUT_SECONDS = 300


In [ ]:
class GoogleLoginManager:
    """使用 Selenium 管理 Google 登入流程的類別。"""

    def __init__(self):
        self.driver = None

    def _initialize_driver(self):
        """初始化 Selenium WebDriver。"""
        _LOGGER.info("正在初始化 Selenium WebDriver...")
        chrome_options = Options()
        # 以下是一些在 Docker 或無頭伺服器中常用的選項，在本地端執行時可以保留
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        
        # 讓 Selenium 自動管理 chromedriver
        # 如果自動下載失敗，您需要手動指定 Service 路徑
        # from selenium.webdriver.chrome.service import Service
        # service = Service(executable_path='/path/to/your/chromedriver')
        # self.driver = webdriver.Chrome(service=service, options=chrome_options)
        
        self.driver = webdriver.Chrome(options=chrome_options)

    def perform_login(self):
        """執行整個登入並擷取 Cookie 的流程。"""
        _LOGGER.info("開始 Google 登入流程...")
        try:
            self._initialize_driver()
        except Exception as e:
            _LOGGER.error(f"初始化 WebDriver 失敗，請確認 Chrome 與 chromedriver 已正確安裝: {e}")
            return

        try:
            # 導向 Google 登入頁面
            login_url = "https://accounts.google.com/ServiceLogin"
            self.driver.get(login_url)

            _LOGGER.info(
                "一個新的 Chrome 瀏覽器視窗已經開啟。請在該視窗中完成 Google 登入流程。"
                f"您有 {LOGIN_TIMEOUT_SECONDS // 60} 分鐘的時間。"
            )

            # 等待使用者登入。成功的登入通常會重新導向到 'myaccount.google.com'。
            # 我們等待網址中包含這個字串。
            WebDriverWait(self.driver, LOGIN_TIMEOUT_SECONDS).until(
                EC.url_contains("myaccount.google.com")
            )

            _LOGGER.info("偵測到登入成功！正在擷取 Cookies...")
            # 增加一個短暫的延遲，確保所有 Cookies 都已設定完成
            time.sleep(5)

            captured_cookies = self.driver.get_cookies()

            if captured_cookies:
                self._save_cookies(captured_cookies)
            else:
                _LOGGER.error("❌ 偵測到登入，但未能擷取到任何 Cookies。")

        except TimeoutException:
            _LOGGER.error(
                f"❌ 登入流程在 {LOGIN_TIMEOUT_SECONDS} 秒後超時。請重試。"
            )
        except Exception as e:
            _LOGGER.error(f"登入過程中發生未預期的錯誤: {e}", exc_info=True)

        finally:
            if self.driver:
                _LOGGER.info("關閉瀏覽器並清理。")
                self.driver.quit()

    def _save_cookies(self, cookies: list[dict]):
        """將擷取到的 Cookies 儲存到檔案。"""
        try:
            with open(COOKIE_PATH, "w") as f:
                json.dump(cookies, f, indent=2)
            _LOGGER.info(f"✅ Cookies 已成功儲存至: {os.path.abspath(COOKIE_PATH)}")
        except IOError as e:
            _LOGGER.error(f"寫入 Cookies 到 {COOKIE_PATH} 失敗: {e}")



# 執行下面的儲存格來啟動整個流程

執行後，會彈出一個新的 Chrome 瀏覽器視窗。請在**那個新視窗**中完成您的 Google 帳號登入（包含兩步驟驗證）。

完成後，程式會自動關閉瀏覽器，並在此 Notebook 的目錄下生成 `google_maps_cookies.json` 檔案。


In [ ]:
# 建立 LoginManager 實例並執行登入
login_manager = GoogleLoginManager()
login_manager.perform_login()
